## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-08-56-06 +0000,nypost,"Arizona couple found blood-spotted glove, rock...",https://nypost.com/2026/02/21/us-news/arizona-...
1,2026-02-21-08-50-35 +0000,nyt,"At A.I. Summit, India Tries to Find a Way Betw...",https://www.nytimes.com/2026/02/21/world/asia/...
2,2026-02-21-08-46-40 +0000,bbc,"Avalanches kill five in Austrian Alps, officia...",https://www.bbc.com/news/articles/c5yqgz2q58lo...
3,2026-02-21-08-44-57 +0000,nyt,Trump Official Backs Russia’s Return to Global...,https://www.nytimes.com/2026/02/21/world/europ...
4,2026-02-21-08-27-54 +0000,nypost,Georgia high school teacher arrested in allege...,https://nypost.com/2026/02/21/us-news/georgia-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,71
225,court,44
224,supreme,40
228,tariffs,35
172,tariff,29


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,319
96,2026-02-20-23-26-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,247
147,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,246
260,2026-02-20-15-03-53 +0000,cbc,Trump imposes new 10% global tariff after U.S....,https://www.cbc.ca/news/world/livestory/scotus...,237
217,2026-02-20-17-23-09 +0000,nypost,Democrats giddy over Trump Supreme Court tarif...,https://nypost.com/2026/02/20/us-news/democrat...,233


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,319,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
260,78,2026-02-20-15-03-53 +0000,cbc,Trump imposes new 10% global tariff after U.S....,https://www.cbc.ca/news/world/livestory/scotus...
92,61,2026-02-20-23-54-09 +0000,latimes,What the Supreme Court's decision to strike do...,https://www.latimes.com/business/story/2026-02...
277,49,2026-02-20-14-01-00 +0000,wsj,Key inflation metrics tracked by the Fed accel...,https://www.wsj.com/economy/central-banking/fe...
304,43,2026-02-20-11-27-40 +0000,nypost,Inside covert police operation to arrest ex-Pr...,https://nypost.com/2026/02/20/world-news/insid...
244,42,2026-02-20-15-58-01 +0000,nypost,The government agency relying on Wall Street t...,https://nypost.com/2026/02/20/business/develop...
273,38,2026-02-20-14-34-29 +0000,nypost,NFL commish thought Bad Bunny’s halftime show ...,https://nypost.com/2026/02/20/business/nfls-ro...
250,32,2026-02-20-15-40-27 +0000,48hills,Screen Grabs: End of days at Fear and Faith Ho...,https://48hills.org/2026/02/screen-grabs-end-o...
36,31,2026-02-21-03-02-20 +0000,bbc,"Anna Murdoch-Mann, writer and former News Corp...",https://www.bbc.com/news/articles/cp8150xj36wo...
17,31,2026-02-21-05-12-15 +0000,nypost,Two more skiers killed in Lake Tahoe — after C...,https://nypost.com/2026/02/21/us-news/lake-tah...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
